In [61]:
from datetime import datetime
import os, time, re, urllib.parse
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [62]:
# Selenium options required to create a 'headless' browser
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1280,800")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.37")

driver = webdriver.Chrome(options=options)

In [63]:
def is_plus_code(code):
    # Regex pattern for Plus Codes like "7FG9V3F5+X2" or "2HVW+G8"
    pattern = r'^[2-9A-HJ-NP-Z]{4,7}\+[2-9A-HJ-NP-Z]{2,}$'
    return re.match(pattern, code) is not None

def get_maps_url(origin, destination):
    origin = urllib.parse.quote(origin)
    destination = urllib.parse.quote(destination)
    url = f"https://www.google.com/maps/dir/{origin}/{destination}/data=!3e0"
    return url

def avg_speed(duration, distance):
    distance = float(distance.split(' ')[0])
    d = duration.split(' ')
    duration = int(d[0])*60 if 'hr' in d else 0
    duration += int(d[-2]) if 'min' in d else 0
    return distance/duration*60

def get_traffic_report(origin, destination, mode='car'):
    modes = {'bike': "\ue9f9", 'car': "\ue531", 'transit': "\ue535"}
    if mode in modes.keys():
        mode = modes[mode]
    else:
        mode = modes['car']

    maps_url = get_maps_url(origin, destination)
    print(maps_url)
    driver.get(maps_url)

    routes = driver.find_elements(By.CSS_SELECTOR, "div[data-trip-index]")
    time_taken = routes[0].text.split("\n")[1]
    for route in routes:
        if mode in route.text:
            time_taken = route.text.split("\n")[1]
            distance = route.text.split("\n")[2]
            break

    return time_taken, distance

In [64]:
routes = [
    {"route_code": "2HVW+G8|XJXR+WG",
     "origin": "Bethel AG Church, Hebbal Kempapura, Bengaluru",
     "destination": "SMVT Railway Station, Bengaluru"},

    {"route_code": "XHCW+W2|3H3V+J5",
     "origin": "JW Marriott Hotel, Bengaluru",
     "destination": "Hyatt Centric Hebbal, Bengaluru"},

    {"route_code": "WGH7+QP|XHJ7+MG",
     "origin": "Cult RR Nagar Fitness Center, Bengaluru",
     "destination": "Lulu Mall, Bengaluru"},

    {"route_code": "XMXH+X3J|WMGQ+34",
     "origin": "K R Pura Metro Station, Bengaluru",
     "destination": "RMZ Ecoworld, Bellandur, Bengaluru"},

    {"route_code": "WH5F+26|WJ8X+F5W",
     "origin": "Jaya Prakash Nagar Metro Station, Bengaluru",
     "destination": "Hemavathi Park, HSR Layout, Bengaluru"},

    {"route_code": "XPC7+72|XM33+J3",
     "origin": "The Rameshwaram Cafe @ Brookfield, Bengaluru",
     "destination": "Gawky Goose, Wind Tunnel Rd, Bengaluru"},

    {"route_code": "XJG4+7J|5PX4+HQ",
     "origin": "MG Road Metro Station, Bengaluru",
     "destination": "Kempegowda International Airport Bengaluru"}
]

In [65]:
today = datetime.now()
df = pd.DataFrame()

for route in routes:
    origin = route["origin"]
    destination = route["destination"]
    o_code, d_code = route["route_code"].split("|")
    travel_time, travel_distance = get_traffic_report(origin, destination)

    new_row = {
        "date": today.date(),
        "time": today.time().strftime("%H:%M"),
        "route_code": route["route_code"],
        "from": origin,
        "to": destination,
        "duration": travel_time,
        "distance": travel_distance,
        "avg_speed": avg_speed(travel_time, travel_distance),
    }

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

# driver.quit()
# out_file = "blr_traffic_times.csv"
# df.to_csv(out_file, mode="a", header=header, index=False)

https://www.google.com/maps/dir/Bethel%20AG%20Church%2C%20Hebbal%20Kempapura%2C%20Bengaluru/SMVT%20Railway%20Station%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/JW%20Marriott%20Hotel%2C%20Bengaluru/Hyatt%20Centric%20Hebbal%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/Cult%20RR%20Nagar%20Fitness%20Center%2C%20Bengaluru/Lulu%20Mall%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/K%20R%20Pura%20Metro%20Station%2C%20Bengaluru/RMZ%20Ecoworld%2C%20Bellandur%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/Jaya%20Prakash%20Nagar%20Metro%20Station%2C%20Bengaluru/Hemavathi%20Park%2C%20HSR%20Layout%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/The%20Rameshwaram%20Cafe%20%40%20Brookfield%2C%20Bengaluru/Gawky%20Goose%2C%20Wind%20Tunnel%20Rd%2C%20Bengaluru/data=!3e0
https://www.google.com/maps/dir/MG%20Road%20Metro%20Station%2C%20Bengaluru/Kempegowda%20International%20Airport%20Bengaluru/data=!3e0


In [66]:
display(df)

,date,time,route_code,from,to,duration,distance,avg_speed
0,2025-07-26,19:15,2HVW+G8|XJXR+WG,"Bethel AG Church, Hebbal Kempapura, Bengaluru","SMVT Railway Station, Bengaluru",42 min,9.9 km,14.142857
1,2025-07-26,19:15,XHCW+W2|3H3V+J5,"JW Marriott Hotel, Bengaluru","Hyatt Centric Hebbal, Bengaluru",33 min,10.5 km,19.090909
2,2025-07-26,19:15,WGH7+QP|XHJ7+MG,"Cult RR Nagar Fitness Center, Bengaluru","Lulu Mall, Bengaluru",43 min,9.3 km,12.976744
3,2025-07-26,19:15,XMXH+X3J|WMGQ+34,"K R Pura Metro Station, Bengaluru","RMZ Ecoworld, Bellandur, Bengaluru",27 min,11.1 km,24.666667
4,2025-07-26,19:15,WH5F+26|WJ8X+F5W,"Jaya Prakash Nagar Metro Station, Bengaluru","Hemavathi Park, HSR Layout, Bengaluru",32 min,10.2 km,19.125000
5,2025-07-26,19:15,XPC7+72|XM33+J3,"The Rameshwaram Cafe @ Brookfield, Bengaluru","Gawky Goose, Wind Tunnel Rd, Bengaluru",33 min,9.4 km,17.090909
6,2025-07-26,19:15,XJG4+7J|5PX4+HQ,"MG Road Metro Station, Bengaluru",Kempegowda International Airport Bengaluru,59 min,34.2 km,34.779661
